# **Gradient Boosting Machines**

Noah Rubin

Self Study - June 2021

## **Gradient Boosting**

#### **Intro**

* While random forests remains are a low variance model when compared to decision trees, all the trees are built independently, hence they are not built off the results of the previous tree
* Gradient boosting
* What if we could build an ensemble of trees but have each subsequent tree build off the errors made in the one before it? 
* [Gradient Boosting Machines (GBM)](https://en.wikipedia.org/wiki/Gradient_boosting) is a non-parametric model that aims to resolve this problem using partial derivatives, hence the name "gradient boosting"
* The idea is similar to algorithms like [AdaBoost](https://en.wikipedia.org/wiki/AdaBoost#:~:text=AdaBoost%2C%20short%20for%20Adaptive%20Boosting,G%C3%B6del%20Prize%20for%20their%20work.&text=AdaBoost%20is%20adaptive%20in%20the,instances%20misclassified%20by%20previous%20classifiers.), gradient boosting leverages the power of many "weak learners" to create a single single strong learner (in an iterative fashion).
* It can deal with data measured in different units but like other tree models like random forest and decision trees it can't extrapolate


#### **Algorithm Details**

<span style="color:blue"><ins><b>Step 0: Define our dataset and a loss function that we can differentiate</b></ins><a name="GradientBoosting"></a></span>

So lets say we have a training dataset with $n$ observations $\{(x_i, y_i)\}^{n}_{i = 1}$ and a differentiable loss function $L(y_i, F(x)).$

---
Our differentiable loss function can be defined as 
$$L(y_i, F(x)) = \frac{1}{2}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \frac{1}{2}\sum_{i=1}^{n} (y_i - F(x))^2$$ 


which ultimately follows the same framework as OLS where we aim to minimise the sum of the squared residuals In fact, the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) notes that the default parameter for the loss function to be used using gradient boosting is "squared error".


#### <span style="color:blue"><b><ins>Step 1: Initialise the model with a constant value</ins></b> $F_0(x) = \text{argmin}_\gamma \sum_{i=1}^{n} L(y_i, \gamma)$</span>.

* The $L(y_i, \gamma)$ is just our original loss function but instead of $F(x)$ we have $\gamma$ because we need an initial constant which will be our prediction to build on. It acts as our initial educated guess.
* To do this we find the optimal constant $\gamma$ that minimises the sum of squared residuals (our loss function)

---

Now if $L(y_i, \gamma) = \frac{1}{2}\sum_{i=1}^{n} (y_i - \gamma)^2$, then it is true that $\frac{\partial{L(y_i, \gamma)}}{\partial\gamma} = -\sum_{i=1}^{n} (y_i - \gamma)$   by the chain rule. Setting this quantity equal to 0 to find the stationary point we obtain the result that

$$-\sum_{i=1}^{n} (y_i - \gamma) = 0$$

Dividing by $-1$ on both sides

$$\sum_{i=1}^{n} (y_i - \gamma) = 0$$

Taking the sum of each term individually,

$$\sum_{i=1}^{n} y_i - \sum_{i=1}^{n} \gamma = 0$$

Recognising that $\sum_{i=1}^{n} y_i = {n}\bar{y}$, and that we are taking $n$ copies of gamma,

$$n\bar{y} - n\gamma = 0$$

Dividing through by $n$ and moving the gamma term to other side,

$$\gamma = \bar{y}$$

This means that the intitial predicted value, representing the single isolated leaf, is $F_0(x) = \gamma = \bar{y} = \text{The mean of the Life_exp column}$

This is a minimum since $L$ is convex and $\frac{\partial ^2 L(y_i, \gamma)}{\partial\gamma^2} = n, \text{and }n > 0$ 

#### <span style="color:blue"><ins>**Step 2: Start Building Trees**</ins></span>

* Setup a for loop: for $m = 1$ to $M$. This means we want to make $M$ trees. For example when we are on the 24th tree, $m = 24$. When we are on the 67th tree, $m = 67$. If we decide to build 200 trees then we have $M = 200$ trees
* With our $m^{th}$ tree we want to:

#### <span style="color:red">a) Compute $\text{}r_{im}$</span>

$$r_{im} = -\big{[}\frac{\partial{L(y_i, F(x_i))}}{\partial{F(x_i)}}\big{]}_{F(x) = F_{(m-1)}(x)} \forall i = 1, 2, ...n.$$


$r_{im}$ is referring to the $i^{th}$ "pseudo residual" for the $m^{th}$ tree we are trying to build. Inside the square brackets is just the partial derivative of our loss function with respect to $F(x_i)$ (the predicted value) for each individual observation $i = 1, 2, ...n$ 

As we are looking to find the derivative of the loss function and apply it to each individual observation, our loss for each row can be expressed without the summation that was present originally. This means that for this step of the process we can set 

$$L(y_i, F(x_i)) = \frac{1}{2}(y_i - F(x_i))^2$$

So the partial derivative with respect to $F(x_i)$ must then be

$$\frac{\partial{L(y_i, F(x_i))}}{\partial{F(x_i)}} = -(y_i - F(x_i))$$

If we look left of the big square brackets in our $r_{im}$ equation, this will cancel the minus we just got to give us a "pseudo-residual" of $(y_i - F(x_i))$. With this result, substitute in $F(x) = F_{m-1}(x)$ which makes sense because for the first tree where $m = 1$, it is taking into account our 0th tree (the single leaf) which was our initial prediction $\gamma = \bar{y}$. Once evaluated, the whole thing is just the pseudo-residual for row $i$ in our $m^{th}$ tree we are trying to build. It is a [pseudo residual](https://www.youtube.com/watch?v=2xudPOBz-vs) because it would have been different if we decided not to have the $\frac{1}{2}$ out the front of the loss function. But this isn't a problem because the same process is used to minimise the sum of squared residuals and the sum of squared residuals multiplied by some constant.

---
#### <span style="color:red">b) Fit a regression tree to the pseudo-residual and create terminal regions $R_{j,m}$ for $j = 1.....J_m$</span>

* This is the part where we fit a regression tree, with our newly formed pseudo-residual column $r_{i1}$ acting as the dependent variable. We build a tree where instead of trying to predict life expectancy, we predict the new pseudo-residuals that were just calculated for each observation in the last step
* As all regression trees have leaf nodes, we can say that the $j^{th}$ leaf in our $m^{th}$ tree can be expressed as $R_{j,m}.$
* The regression tree is built using traditional methods such as minimising mean squared error or mean absolute error if we choose
* In summary, this part is about fitting a regression tree (using mae or mse criterion) to the residuals and assigning $j$ index numbers to the leaves. We call a leaf in a tree $R_{j,m}$ which means that we are referring to the the $j^{th}$ leaf in our $m^{th}$ tree. We have a total of $J_m$ leaves in our $m^{th}$ tree

---

#### <span style="color:red">c) For $j = 1.....J_m$ compute $\gamma_{jm} = \text{argmin}_\gamma \sum_{x_i \in R_{jm}}^{} L(y_i, F_{m-1}(x_i) +  \gamma)$</span>


* From the regression tree that was just built, what do we do if many values fall in the same leaf node? It turns out we take the average, but this can be shown
* The output value for each leaf is the value for gamma that minimises the summation above, which is ultimately very similar to step 1.
* However there are differences in what we are summing over. This new formula in this heading only sums over very particular values whereas the one before summed over all the $n$ samples
* The $x_i \in R_{j,m}$ part implies that if only row 56 got put into the first leaf of our first tree $R_{1,1}$, then only observation number 56 is used to calculate the output value for $R_{1,1}$
* But in the case where multiple observations get sent to the same leaf, we need calculate the output value for when this happens
* The output value for a leaf with multiple values in it is just the mean of all of the residual values that fell in that leaf
* Also note that $F_{m-1}(x_i)$ ensures that we take the previous prediction into account. In the initial summation in step 1, there was no previous prediction as that step was about initialising a constant value to start the process for gradient boosting.

Our loss function becomes $L(y_i, F_{m-1}(x_i) + \gamma_{j, m}) = \frac{1}{2}\sum_{i=1}^{k} (y_i - F_{m-1}(x_i) - \gamma)^2$. If we assume that there are a total of $k$ elements in a terminal region $R_{jm}.$ 

Differentiating our loss function and setting the quantity equal to zero we get

$$-\sum_{i=1}^{k}(y_i - F_{m-1}(x_i) - \gamma) = 0$$

Dividing both sides by -1,

$$\sum_{i=1}^{k}(y_i - F_{m-1}(x_i) - \gamma) = 0$$

Splitting the summation into two parts

$$\sum_{i=1}^{k}(y_i - F_{m-1}(x_i)) - \sum_{i=1}^{k} \gamma = 0$$

Noticing that we have $k$ copies of $\gamma$, moving this term to the other side

$$\sum_{i=1}^{k}(y_i - F_{m-1}(x_i)) = k\gamma$$

Dividing both sides by $k$ we obtain the result 

$$\gamma_{j, m} = \frac{\sum_{i=1}^{k}(y_i - F_{m-1}(x_i))}{k}$$

Which implies that the output of the $j^{th}$ leaf in our $m^{th}$ tree is the sum of all the residual values $y_i - F_{m-1}(x_i)$ in a leaf $j$, divided by the amount of values $k$ in the particular leaf.

Summary

* This step was about finding the output value $\gamma_{j, m}$ for a particular leaf node for when we build our $m^{th}$ regression tree
* The output for a leaf will be the mean of all the values that were in that leaf
---

<span style="color:red">d) Update $F_m(x) = F_{m-1}(x) + \alpha \sum_{j=1}^{J_m} \gamma_{j,m} I(x \in R_{jm})$</span>


* In this step we update our predicted valiue for life expectancy using our initial estimate, plus the results from the first tree we built
* As this is the first time we are passing through step two, the new prediction will be called $F_1(x)$
* So this $F_1(x)$ is given by $F_0(x)$ (from step one) plus a learning rate (which is a hyperparameter that we can tune) multiplied by the output value for a particular leaf in our $m^{th}$ regression tree
* If we set alpha = 0.1 the small learning rate reduces the effect each tree has on the final prediction, and this improves accuracy in the long run
* "Empirically it has been found that using small learning rates (such as $\alpha < 0.1$) yields dramatic improvements in models' generalisation ability over gradient boosting without shrinking ($\alpha = 1$) However, it comes at the price of increasing computational time both during training and querying: lower learning rate requires more iterations." - Wikipedia 


In [1]:
# Python files
import data_prep
import helper_funcs

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

%matplotlib inline

In [2]:
train = pd.read_csv('../datasets/train_updated.csv')
test = pd.read_csv('../datasets/test_updated.csv')

# Split data
to_drop = ['Country', 'HDI', 'Life_exp']

X_train = train.drop(to_drop, axis='columns')
X_test = test.drop(to_drop, axis='columns')

y_train = train['Life_exp']
y_test = test['Life_exp']

# Preprocessing & Model Building

In [3]:
model_pipeline = data_prep.create_pipeline(GradientBoostingRegressor())

In [4]:
# Hyperparameters for knn imputer that we can try
param_grid = {
    'imputation__n_neighbors': np.arange(5, 36, 2), 
    'imputation__weights': ['uniform', 'distance'],
    'model__n_estimators': np.arange(100, 300, 5),
    'model__min_samples_split': np.arange(10, 40, 1),
    'model__min_samples_leaf': np.arange(3, 30, 1),
    'model__max_depth': np.arange(3, 8),
    'model__criterion': ['squared_error', 'absolute_error']
}

# Get the best hyperparameters for each model and use that in the final model
final_model, best_params = data_prep.randomised_search_wrapper(X_train,
                                                               y_train,
                                                               model_pipeline, 
                                                               param_grid, 
                                                               cv=10,
                                                               n_iter=15)


Best Parameters were...

model__n_estimators had optimal value as: 180
model__min_samples_split had optimal value as: 29
model__min_samples_leaf had optimal value as: 8
model__max_depth had optimal value as: 7
model__criterion had optimal value as: squared_error
imputation__weights had optimal value as: uniform
imputation__n_neighbors had optimal value as: 29

The fitted model just initialised and fit now has all these parameters set up


## Evaluation Metrics

In [5]:
r2, mse, rmse, mae = helper_funcs.display_regression_metrics(y_test, final_model.predict(X_test))

All metrics are in terms of the unseen test set

R^2 = 0.9862984406482037
Mean Squared Error = 1.1100456092359872
Root Mean Squared Error = 1.0535870202484403
Mean Absolute Error = 0.7315371637506867


## Save Model

In [7]:
import joblib
joblib.dump(final_model, './saved_models/GradientBoosting.joblib')

['./saved_models/GradientBoosting.joblib']